# Project 4

In [1]:
!pip install CFEDemands --upgrade
!pip install ConsumerDemands
!pip install oauth2client
!pip install eep153_tools
!pip install gnupg
#!pip install dvc

Requirement already up-to-date: CFEDemands in /opt/conda/lib/python3.9/site-packages (0.4.1)


In [2]:
from eep153_tools.sheets import read_sheets
import numpy as np
import pandas as pd

### Data

In [3]:
InputFiles = {'Expenditures':('1mZcHvSD55wGCicdqxbEnioE3AOXM_pVb6zVwHhtBzSk','Expenditures'),
              'Consumption':('1RT5AfbZbWx-CmNaOHLp7aD9qEjgjiBv8lZmPexuKEWg','Consumption'),
              'HH Characteristics':('1mZcHvSD55wGCicdqxbEnioE3AOXM_pVb6zVwHhtBzSk','HH Characteristics'),
              'FCT': ('1DjpLsyhUbQ-giGcTS_GMrDuW_wG3VDrYH1p-FanKxxs','Codes')}

#### Function to read in sheets:

In [4]:
def get_clean_sheet(key,sheet=None):
    dfs = read_sheets(key)
    if sheet is not None:
        df = dfs[sheet]
    else:
        df = dfs.values()[0] 
    df.columns = [c.strip() for c in df.columns.tolist()]
    df = df.loc[:,~df.columns.duplicated(keep='first')]   
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)
    df = df.loc[~df.index.duplicated(), :]
    return df

#### Expenditure Table

In [5]:
x = get_clean_sheet(InputFiles['Expenditures'][0],
                    sheet=InputFiles['Expenditures'][1])

if 'm' not in x.columns:
    x['m'] = 1

x = x.set_index(['j','t','m'])
x.columns.name = 'i'

x = x.apply(lambda x: pd.to_numeric(x,errors='coerce'))
x = x.replace(0,np.nan)

# Log of expenditure table (x); 
# log table (y) will be used for later estimation:
y = np.log(x)

Key available for students@eep153.iam.gserviceaccount.com.


#### Household Characteristics Table

In [6]:
z = get_clean_sheet(InputFiles['HH Characteristics'][0],
                    sheet=InputFiles['HH Characteristics'][1])

if 'm' not in z.columns:
    z['m'] = 1

z = z.set_index(['j','t','m'])
z.columns.name = 'k'

z = z.apply(lambda x: pd.to_numeric(x,errors='coerce'))

Key available for students@eep153.iam.gserviceaccount.com.


#### Consumption Quantities Table

In [7]:
q = get_clean_sheet(InputFiles['Consumption'][0],
                    sheet=InputFiles['Consumption'][1])

if 'm' not in q.columns:
    q['m'] = 1

q = q.set_index(['j','t','m','u'])
q.columns.name = 'i'

q = q.apply(lambda x: pd.to_numeric(x,errors='coerce'))
q = q.replace(0,np.nan)

Key available for students@eep153.iam.gserviceaccount.com.


#### Food Conversion Table

In [8]:
fct = get_clean_sheet(InputFiles['FCT'][0],sheet=InputFiles['FCT'][1])

fct.index.name = 'i'
foods = fct['Food description']
fct = fct.apply(lambda x: pd.to_numeric(x,errors='coerce'))
fct['Food description'] = foods
fct = fct.set_index('Food description')

Key available for students@eep153.iam.gserviceaccount.com.


## Estimate Demand System

In [9]:
# Create reduced form
import cfe

result = cfe.Result(y=y,z=z)

Missing dependencies for OracleDemands.


In [10]:
# run this to be able to run the next few lines:
result.get_reduced_form()

/opt/conda/lib/python3.9/site-packages/cfe/estimation.py:447: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X=rhs.append(znil.join(timednil))
/opt/conda/lib/python3.9/site-packages/cfe/estimation.py:451: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  myb,mye=ols(X,lhs.append(ynil),return_se=False,return_v=False,return_e=True)
/opt/conda/lib/python3.9/site-packages/cfe/estimation.py:447: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X=rhs.append(znil.join(timednil))
/opt/conda/lib/python3.9/site-packages/cfe/estimation.py:451: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  myb,mye=ols(X,lhs.append(ynil),return_se=False,return_v=False,retur

### Delta Values

In [11]:
result.delta.to_dataframe().unstack('k')

delta                                \
k                                      M 0-3     M 4-8    M 9-13   M 14-18   
i                                                                            
beans dry                          -0.020105  0.091101  0.069991  0.131634   
beans fresh                         0.003502  0.138232  0.049423  0.043303   
beef                                0.012622  0.025357  0.076339  0.081080   
bread                              -0.010931 -0.010405  0.039982  0.015581   
cabbages                            0.014337 -0.007694  0.035684  0.098675   
cassava dry/flour                   0.240597  0.163090  0.237445  0.090692   
cassava fresh                      -0.003535  0.006866  0.158522  0.094510   
chicken                             0.005440 -0.043764  0.015580  0.035094   
coffee                              0.022706 -0.057133  0.028877  0.008619   
cooking oil                        -0.029436 -0.006941  0.021453  0.052854   
dodo                               -0.064294  0.010363  0.145474  0.102694   
dry/smoked fish                    -0.037726  0.043358  0.094554  0.125807   
eggs                               -0.071013  0.041225  0.006218  0.000555   
expenditure in restaurants on food -0.094559  0.037472  0.151278  0.025689   
fresh fish                          0.020568  0.080817  0.028284  0.073189   
fresh milk                         -0.001583 -0.002583 -0.014292 -0.023467   
goat meat                          -0.052198  0.101357  0.009729  0.055356   
ground nuts pounded                 0.033751  0.022292  0.009320  0.044031   
irish potatoes                      0.086792  0.069802  0.115015  0.128570   
maize cobs                         -0.064494 -0.044831  0.043807  0.182292   
maize flour                         0.009996  0.112082  0.142091  0.075552   
mangoes                            -0.012302  0.032207  0.053649 -0.052200   
margarine,butter                   -0.015707  0.022894 -0.081790 -0.020390   
matooke_1                          -0.017372  0.179616  0.051870  0.053682   
millet                              0.096938  0.067989  0.001464  0.142472   
onions                             -0.051036 -0.027855  0.030258  0.048141   
oranges                            -0.050278  0.097141  0.035387  0.028289   
other vegetables                    0.012330 -0.046587  0.078986  0.052800   
passion fruits                      0.007164 -0.067459  0.028914 -0.094513   
rice                                0.041959  0.089870  0.105172  0.099908   
salt                                0.072773  0.065634  0.082439  0.087910   
soda                               -0.011965 -0.063974  0.055411 -0.049988   
sugar                               0.012851  0.059009  0.045291  0.091241   
sweet bananas                       0.011865  0.005062  0.000327  0.025421   
sweet potatoes fresh                0.110313  0.127481  0.137242  0.100394   
tea                                -0.012445  0.016257  0.036033  0.041242   
tomatoes                           -0.040840 -0.022914  0.018166  0.050890   

                                                                            \
k                                    M 19-30   M 31-50     M 51+     F 0-3   
i                                                                            
beans dry                           0.092998  0.126748  0.192872  0.035235   
beans fresh                         0.140382  0.256491  0.353082  0.098333   
beef                                0.114490  0.197677  0.197012  0.004936   
bread                               0.133572  0.290906  0.142049  0.021215   
cabbages                            0.061728  0.060672  0.079951  0.038930   
cassava dry/flour                   0.106282  0.175328  0.190029  0.068200   
cassava fresh                       0.057267  0.043634  0.019687  0.003492   
chicken                             0.125162  0.132872  0.148593 -0.003726   
coffee                              0.098526  0.060754  0.42

### Alpha Values

In [12]:
result.a.to_dataframe().unstack('i')

a                                                              \
i      beans dry beans fresh      beef     bread  cabbages cassava dry/flour   
t    m                                                                         
2005 1  6.664045    6.166651  7.596403  6.531700  5.987894          7.000972   
2009 1  7.177776    6.660552  8.087632  7.200124  6.284722          7.403977   
2010 1  7.361983    6.874905  8.165703  7.073217  6.369316          7.427036   
2011 1  7.436129    7.184067  8.397829  7.388194  6.608653          7.770406   

                                                      ...                   \
i      cassava fresh   chicken    coffee cooking oil  ... other vegetables   
t    m                                                ...                    
2005 1      6.561716  8.220995  4.586281    6.181261  ...         5.622420   
2009 1      6.985613  8.733654  4.799347    6.605940  ...         6.134787   
2010 1      6.993906  8.798743  4.734974    6.763708  ...         6.100634   
2011 1      7.339227  9.207515  5.074747    6.940089  ...         6.475021   

                                                                             \
i      passion fruits      rice      salt      soda     sugar sweet bananas   
t    m                                                                        
2005 1       6.412390  6.868322  4.370730  7.055358  6.854877      5.973993   
2009 1       6.691317  7.375543  5.028736  7.325378  7.196572      6.407480   
2010 1       6.631896  7.440398  4.963925  7.503323  7.370396      6.422730   
2011 1       6.896860  7.743197  5.120522  7.765180  7.461896      6.816289   

                                                 
i      sweet potatoes fresh       tea  tomatoes  
t    m                                           
2005 1             6.627925  4.610117  5.958305  
2009 1             7.068341  4.678707  6.417172  
2010 1             7.447756  4.683057  6.427528  
2011 1             7.318681  4.936173  6.622662  

[4 rows x 37 columns]

### Beta Values

In [13]:
result.get_beta(as_df=True)

i
beans dry                             0.255622
beans fresh                           0.381523
beef                                  0.384351
bread                                 0.568417
cabbages                              0.357247
cassava dry/flour                     0.124855
cassava fresh                         0.313435
chicken                               0.322282
coffee                                0.632619
cooking oil                           0.483203
dodo                                  0.452957
dry/smoked fish                       0.351626
eggs                                  0.420556
expenditure in restaurants on food    0.460684
fresh fish                            0.412433
fresh milk                            0.567437
goat meat                             0.360544
ground nuts pounded                   0.367467
irish potatoes                        0.388275
maize cobs                            0.356432
maize flour                           0.285216
mangoes    